In [1643]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

In [1644]:
# get the data from last 60 days until the present
# dataF = yf.download("^NDX", start="2023-10-25", end="2023-12-21", interval='15m')
# dataF = yf.download("EURJPY=X", period="60d", interval='15m')
dataF = yf.download("^NDX", period="730d", interval='1h')
# dataF = yf.download("^SPX", period="500d", interval='1h')
# dataF = yf.download("AAPL", period="730d", interval='1h')
# dataF = yf.download("^NDX", period="50d", interval='5m')
# dataF = yf.download("EURUSD=x", period="50d", interval='5m')
# dataF = yf.download("EURJPY=X", period="50d", interval='30m') # <-- this is the one
# dataF = yf.download("EURJPY=X", period="365d", interval='1h')# <-- this is the one
# dataF = yf.download("EURJPY=X", period="500d", interval='1h')# <--------- this is the one
# dataF = yf.download("EURUSD=X", period="365d", interval='1h') # XXXX Horrible
# dataF = yf.download("USDJPY=X", period="365d", interval='1h') <-- not too bad

# dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


In [1645]:
df = pd.DataFrame(dataF)

# use df index, convert DateTime to  a column instead of index
df.reset_index(inplace=True)

# delete Adj Close
df = df.drop(['Adj Close'], axis=1)

# rename Datetime to "Gmt time"
df = df.rename(columns={'Datetime':'Gmt time'})

# set gmt time as index
# df.set_index('Gmt time', inplace=True)

df=df[df.High!=df.Low]
df.reset_index(inplace=True)
df.head()

df

,index,Gmt time,Open,High,Low,Close,Volume
0,0,2021-02-01 09:30:00-05:00,13070.278320,13116.041992,12983.939453,13096.955078,0
1,1,2021-02-01 10:30:00-05:00,13097.088867,13165.235352,13058.913086,13164.789062,81692783
2,2,2021-02-01 11:30:00-05:00,13165.610352,13254.970703,13160.590820,13251.634766,72810373
3,3,2021-02-01 12:30:00-05:00,13252.358398,13279.831055,13230.505859,13264.542969,58294555
4,4,2021-02-01 13:30:00-05:00,13264.546875,13274.939453,13237.343750,13268.390625,56453341
...,...,...,...,...,...,...,...
5082,5082,2023-12-22 11:30:00-05:00,16800.720703,16812.632812,16773.761719,16791.285156,61101860
5083,5083,2023-12-22 12:30:00-05:00,16790.691406,16818.531250,16786.503906,16816.464844,47687476
5084,5084,2023-12-22 13:30:00-05:00,16815.724609,16820.718750,16718.250000,16719.267578,61637204
5085,5085,2023-12-22 14:30:00-05:00,16718.931641,16791.492188,16703.902344,16789.730469,64558894


In [1646]:
import pandas_ta as ta
df["MACD"]=ta.macd(df.Close)['MACD_12_26_9']
df["MACD_HIST"]=ta.macd(df.Close)['MACDh_12_26_9']
df["MACD_SIGNAL"]=ta.macd(df.Close)['MACDs_12_26_9']
df["ADX"]=ta.adx(df.High, df.Low, df.Close)['ADX_14']
df["20_MA"] = ta.ema(df.Close, length=20)
df["50_MA"] = ta.ema(df.Close, length=50)
df["100_MA"] = ta.ema(df.Close, length=100)
df["200_MA"]= ta.ema(df.Close, length=200)
df['EMA']=ta.ema(df.Close, length=200)#sma ema
df['EMA2']=ta.ema(df.Close, length=150)#sma ema
df['EMA3']=ta.ema(df.Close, length=100)#sma ema
df['EMA4']=ta.ema(df.Close, length=50)#sma ema
df['EMA5']=ta.ema(df.Close, length=20)#sma ema
df['RSI']=ta.rsi(df.Close, length=12)

# df["HA_OPEN"]=ta.ha(df.Open, df.High, df.Low, df.Close)['HA_open']
# df["HA_CLOSE"]=ta.ha(df.Open, df.High, df.Low, df.Close)['HA_close']
my_bbands = ta.bbands(df.Close, length=14, std=2.0)
df=df.join(my_bbands)

In [1647]:
def addemasignal(df):
    emasignal = [0]*len(df)
    for i in range(0, len(df)):
        if df.EMA3[i]>df.EMA[i]:
            emasignal[i]=2
        elif df.EMA3[i]<df.EMA[i]:
            emasignal[i]=1
        if (df.EMA4[i]<df.EMA2[i]):
            emasignal[i]=0
    df['EMASignal'] = emasignal
addemasignal(df)

In [1648]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if  df.Close[i]<=df['BBL_14_2.0'][i] and df.EMASignal[i]==2:
            ordersignal[i]=df.Close[i]-df.Close[i]*percent
        elif df.Close[i]>=df['BBU_14_2.0'][i] and df.EMASignal[i]==1:
            ordersignal[i]=df.Close[i]+df.Close[i]*percent
    df['ordersignal']=ordersignal
    
addorderslimit(df, 0.000)

In [1649]:
df['index_entry'] = df.index
# # copy gmt time into a enw column
df['Date'] = df['Gmt time']

# # copy the index to a new col

# # set it as the index
df.set_index('Date', inplace=True)



In [1650]:
df[df.ordersignal!=0]

,index,Gmt time,Open,High,Low,Close,Volume,MACD,MACD_HIST,MACD_SIGNAL,...,EMA5,RSI,BBL_14_2.0,BBM_14_2.0,BBU_14_2.0,BBB_14_2.0,BBP_14_2.0,EMASignal,ordersignal,index_entry
Date,,,,,,,,,,,,,,,,,,,,,
2021-04-05 09:30:00-04:00,301,2021-04-05 09:30:00-04:00,13435.281250,13528.614258,13420.811523,13526.987305,0,122.471651,27.075582,95.396069,...,13193.909831,77.780124,13013.615333,13246.339983,13479.064634,3.513796,1.102960,1,13526.987305,301
2021-04-14 15:30:00-04:00,356,2021-04-14 15:30:00-04:00,13837.846680,13842.144531,13782.174805,13805.905273,70506423,33.113462,-23.267699,56.381161,...,13863.737421,41.808589,13809.766972,13913.122140,14016.477309,1.485722,-0.018682,2,13805.905273,356
2021-04-19 09:30:00-04:00,371,2021-04-19 09:30:00-04:00,13967.009766,14024.805664,13906.299805,13940.795898,0,41.288429,-8.693597,49.982026,...,13977.487442,45.825845,13955.785695,14009.686802,14063.587910,0.769483,-0.139049,2,13940.795898,371
2021-04-19 10:30:00-04:00,372,2021-04-19 10:30:00-04:00,13939.548828,13954.064453,13874.383789,13934.868164,80803779,33.130742,-13.481027,46.611770,...,13973.428463,45.002702,13941.852880,14006.569196,14071.285513,0.924085,-0.053964,2,13934.868164,372
2021-04-19 11:30:00-04:00,373,2021-04-19 11:30:00-04:00,13934.779297,13935.080078,13860.347656,13867.088867,59446298,20.954938,-20.525465,41.480403,...,13963.300883,36.765149,13901.437825,13998.070033,14094.702242,1.380650,-0.177730,2,13867.088867,373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-04 09:30:00-05:00,4982,2023-12-04 09:30:00-05:00,15838.645508,15843.804688,15720.737305,15727.248047,0,-16.247855,-5.456413,-10.791443,...,15931.120718,31.969313,15745.577276,15903.379604,16061.181931,1.984513,-0.058077,2,15727.248047,4982
2023-12-06 14:30:00-05:00,5001,2023-12-06 14:30:00-05:00,15839.655273,15860.909180,15796.269531,15802.354492,79920356,-15.689324,0.573712,-16.263036,...,15859.721260,38.387510,15816.327299,15856.958078,15897.588856,0.512466,-0.171949,2,15802.354492,5001
2023-12-06 15:30:00-05:00,5002,2023-12-06 15:30:00-05:00,15802.128906,15815.443359,15776.951172,15790.952148,94473866,-19.336829,-2.459034,-16.877795,...,15853.171821,36.789917,15801.019811,15853.523786,15906.027762,0.662363,-0.095875,2,15790.952148,5002


## Visualization

In [1651]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [1652]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[1000:].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl.EMA2, line=dict(color='yellow', width=2), name="EMA2"),        
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'], line=dict(color='blue', width=1), name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'], line=dict(color='blue', width=1), name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
#fig.update(layout_yaxis_range = [300,420])
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=1000, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

In [1653]:
dfpl = df[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [1654]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.03
    mysize = initsize
    risk = 0
    latestEntry = 0
    
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.02
        lastEntryIndex = 0
        
        if len(self.trades)>0:
            
            # close the position if the amount lost exceeds 3% of the account balance
            if (self.position.pl < -1 * self.equity * 0.015):
                self.position.close()
                print("Close position due to loss")
                print("------")
            
            # if the pl is greater than the stop loss amount, then move the stop loss to entry
            if (self.position.pl > self.risk):
                # adjust the stop loss to entry
                self.position.sl = self.latestEntry
        
            if self.data['index_entry'][-1]-self.lastEntryIndex>=10:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
            if self.trades[-1].is_long and self.data.RSI[-1]>=75:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
            elif self.trades[-1].is_short and self.data.RSI[-1]<=25:
                self.trades[-1].close()
                self.lastEntryIndex = self.data['index_entry'][-1]
        
        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:  
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            entry = self.data.Close[-1]
            self.risk = entry - sl1
            self.latestEntry = entry
            self.lastEntryIndex = self.data['index_entry'][-1]
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:         
            sl1 = sl1 = max(self.data.High[-1],self.data.High[-2])*(1+perc)
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            entry = self.data.Close[-1]
            self.risk = sl1 - entry
            self.latestEntry = entry
            self.lastEntryIndex = self.data['index_entry'][-1]
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=50000, margin=1/100, commission=.000)
stat = bt.run()
stat

Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position due to loss
------
Close position

Start                     2021-02-01 09:30...
End                       2023-12-22 15:30...
Duration                   1054 days 06:00:00
Exposure Time [%]                   15.922941
Equity Final [$]                105609.308652
Equity Peak [$]                 106073.511777
Return [%]                         111.218617
Buy & Hold Return [%]               28.101564
Return (Ann.) [%]                   29.495031
Volatility (Ann.) [%]               26.712652
Sharpe Ratio                          1.10416
Sortino Ratio                        2.460049
Calmar Ratio                         2.201169
Max. Drawdown [%]                  -13.399709
Avg. Drawdown [%]                   -1.949544
Max. Drawdown Duration      342 days 19:00:00
Avg. Drawdown Duration       15 days 10:00:00
# Trades                                   96
Win Rate [%]                        52.083333
Best Trade [%]                       4.924135
Worst Trade [%]                      -2.08918
Avg. Trade [%]                    

In [1655]:
bt.plot()

/var/folders/mb/tyl8xvh9689bgk14qr31lc7r0000gn/T/ipykernel_63517/651457420.py:1: UserWarning:

Superimposed OHLC plot matches the original plot. Skipping.



Row(id='115062', ...)